In [1]:
import numpy as np

import ctdna
from ctdna.detection import compute_pval_th, calculate_sensitivity
from ctdna.utils import diameter_cells, cells_diameter, calculate_elimination_rate
import ctdna.settings as settings

In [2]:
# lung cancer
b_lung = 0.14  # cell birth rate
d_lung = 0.136  # cell death rate
q_d_lung = 1.4e-4  # shedding probability

fpr = 0.01  # false positive rate
seq_err = 1e-5  # sequencing error rate

# convert cfDNA half-life time in minutes to an elimination rate per day
t12_cfdna_mins = 30
epsilon = calculate_elimination_rate(t12_cfdna_mins)

# parameters for the gamma-distributed plasma DNA concentrations
dna_conc_gamma_params = settings.FIT_GAMMA_PARAMS

seq_eff = 0.5  # sequencing efficiency
panel_size = 1  # consider exactly one actionable mutation
n_det_muts = 1  # number of called mutations required for detection
n_muts_cancer = n_det_muts  # actionable mutation is present in the cancer cells

# translate tumor diameters [cm] into number of cells
tumor_sizes = np.array([diameter_cells(1), 1e9, diameter_cells(1.5), diameter_cells(2)])

# calculate a threshold to call a mutation such that a given false positive rate is achieved
pval_th = compute_pval_th(
    fpr, panel_size, seq_err, seq_eff, dna_conc_gamma_params, epsilon=epsilon)

# calculate the probability to detect a mutation of tumors with different sizes
det_probs = calculate_sensitivity(
    b_lung, d_lung, q_d_lung, epsilon, n_det_muts, panel_size, n_muts_cancer,
    pval_th=pval_th, dna_conc_gamma_params=dna_conc_gamma_params,
    seq_err=seq_err, seq_eff=seq_eff, tumor_sizes=tumor_sizes)

diameters = [cells_diameter(size) for size in tumor_sizes]
print('Detection probabilities for tumors of sizes: '
      + ', '.join(f'{p:.1%} ({d:.1f} cm; {c:.1e})' for p, c, d in zip(det_probs, tumor_sizes, diameters)))

2020-09-14 12:12:47,518 ctdna.utils:323 INFO: cfDNA half life of 30 mins leads to an elimination rate epsilon of 33.271 per day.
2020-09-14 12:12:47,520 ctdna.detection:107 INFO: Calculating p-val threshold for 1 det. mut panel size 1.0e+00 with seq. err 1.0e-05 and seq. eff 50.0% used every None days for a desired normalized fpr of 1.0e-02 (annual 1.0e-02).
2020-09-14 12:12:47,719 ctdna.detection:241 INFO: Computed a p-value threshold of 3.745e-02 for a panel size of 1 applied every None days for a desired normalized fpr of 1.000e-02 (annual 1.000e-02).
2020-09-14 12:12:48,344 ctdna.detection:581 INFO: 1 called muts required for detection need: mean 1.737, median 2.0 mutant fragments.
2020-09-14 12:12:48,345 ctdna.detection:584 INFO: Sensitivity for tumor size 5.2e+08 (mean 299.6 hGE, 1 called muts): 17.622% (pv 3.745e-02)
2020-09-14 12:12:48,933 ctdna.detection:581 INFO: 1 called muts required for detection need: mean 1.737, median 2.0 mutant fragments.
2020-09-14 12:12:48,934 ctdna.

Detection probabilities for tumors of sizes: 17.6% (1.0 cm; 5.2e+08), 33.6% (1.2 cm; 1.0e+09), 56.0% (1.5 cm; 1.8e+09), 90.7% (2.0 cm; 4.2e+09)
